What I'm Trying top predict for this project is a number of rings on the shell of an Abalone ( A type of mollusk ) they are also called sea ears.
The number of rings is supposed to represent the age.

![Abalone](https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/LivingAbalone.JPG/220px-LivingAbalone.JPG)

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_log_error

In [24]:
train_file = "./train.csv"
test_file = "./test.csv"

train = pd.read_csv(train_file, index_col=0)
test = pd.read_csv(test_file, index_col=0)
print(train.head())
print(test.head())

   Sex  Length  Diameter  Height  Whole weight  Whole weight.1  \
id                                                               
0    F   0.550     0.430   0.150        0.7715          0.3285   
1    F   0.630     0.490   0.145        1.1300          0.4580   
2    I   0.160     0.110   0.025        0.0210          0.0055   
3    M   0.595     0.475   0.150        0.9145          0.3755   
4    I   0.555     0.425   0.130        0.7820          0.3695   

    Whole weight.2  Shell weight  Rings  
id                                       
0           0.1465        0.2400     11  
1           0.2765        0.3200     11  
2           0.0030        0.0050      6  
3           0.2055        0.2500     10  
4           0.1600        0.1975      9  
      Sex  Length  Diameter  Height  Whole weight  Whole weight.1  \
id                                                                  
90615   M   0.645     0.475   0.155        1.2380          0.6185   
90616   M   0.580     0.460   0.160 

In [25]:
encoder = LabelEncoder()

train['Sex'] = encoder.fit_transform(train['Sex'])
test['Sex'] = encoder.transform(test['Sex'])
print(train['Sex'].head())

id
0    0
1    0
2    1
3    2
4    1
Name: Sex, dtype: int32


In [26]:
X = train.drop('Rings', axis=1)
y = train['Rings']

scaler = StandardScaler()
X = scaler.fit_transform(X)
test1 = scaler.transform(test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [27]:
linreg = LinearRegression()

linreg.fit(X_train, y_train)

y_pred = np.round(linreg.predict(X_test))
y_pred[y_pred<1] = 1

In [28]:
rmsle = root_mean_squared_log_error(y_true=y_test, y_pred=y_pred)
print(rmsle)

0.16745761106008042


That's a good baseline i think. would like to get it to at least 0.14 so i can get on the leaderboard. This will be my baseline.

## Model iteration 3

In [29]:
from sklearn.ensemble import GradientBoostingRegressor

In [30]:
param_grid = {
    "n_estimators": [100, 200, 300],  # Number of boosting stages
    "learning_rate": [0.1, 0.05, 0.01],  # Controls step size
    "max_depth": [3, 5, 8],  # Maximum depth of individual trees
    "min_samples_split": [2, 5, 10],  # Minimum samples required for a split
    "min_samples_leaf": [1, 2, 4]   # Minimum samples required at each leaf node
}

gbr = GradientBoostingRegressor()

search = RandomizedSearchCV(gbr, param_grid, cv=5, scoring="neg_mean_squared_error")

search.fit(X, y)

best_model = search.best_estimator_
best_params = search.best_params_

print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 8, 'learning_rate': 0.05}


In [31]:
gbr_final_3 = GradientBoostingRegressor(**best_params)

gbr_final_3.fit(X, y)

y_pred_final_3 = np.round(gbr_final_3.predict(test1))
y_pred_final_3[y_pred_final_3 < 1] = 1

In [32]:
df_final_3 = pd.DataFrame()
df_final_3.index = test.index
df_final_3['Rings'] = y_pred_final_3

print(df_final_3.head())

       Rings
id          
90615   10.0
90616   10.0
90617   10.0
90618   10.0
90619    8.0


In [33]:
df_final_3['Rings'] = df_final_3['Rings'].astype('int')
print(df_final_3.head())

       Rings
id          
90615     10
90616     10
90617     10
90618     10
90619      8


In [34]:
df_final_3.to_csv('submission_3.csv')